In [1]:
from pyspark.sql import SparkSession
import os


In [2]:
spark = SparkSession.builder \
        .appName("app") \
        .master("local[40]") \
        .config("spark.driver.memory", "100g") \
        .config("spark.executor.memory", "100g") \
        .config("spark.jars", "postgresql-42.3.3.jar") \
        .getOrCreate()
#spark.sparkContext.setLogLevel("DEBUG")

23/11/09 09:17:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
spark.sparkContext.setLogLevel("WARN") # ALL, DEBUG, WARN,
#spark.sparkContext.setLogLevel("ALL") # ALL, DEBUG, WARN,

In [3]:
username = os.environ.get('USERNAME', 'lsqb')
password = os.environ.get('PASSWORD', 'lsqb')
dbname = os.environ.get('DBNAME', 'lsqb')
dbhost = os.environ.get('DBHOST', 'postgres')

df_tables = spark.read.format("jdbc") \
    .option("url", f'jdbc:postgresql://{dbhost}:5432/{dbname}') \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "information_schema.tables") \
    .option("user", username) \
    .option("password", password) \
    .load()

for idx, row in df_tables.toPandas().iterrows():
        if row.table_schema == 'public':
            table_name = row.table_name
            df = spark.read.format("jdbc") \
                .option("url", f'jdbc:postgresql://{dbhost}:5432/{dbname}') \
                .option("driver", "org.postgresql.Driver") \
                .option("dbtable", table_name) \
                .option("user", username) \
                .option("password", password) \
                .load()
    
            print(table_name)
            print(df.show())
            df.createOrReplaceTempView(table_name)

Py4JJavaError: An error occurred while calling o39.load.
: org.postgresql.util.PSQLException: FATAL: password authentication failed for user "lsqb"
	at org.postgresql.core.v3.ConnectionFactoryImpl.doAuthentication(ConnectionFactoryImpl.java:646)
	at org.postgresql.core.v3.ConnectionFactoryImpl.tryConnect(ConnectionFactoryImpl.java:180)
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:235)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:49)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:223)
	at org.postgresql.Driver.makeConnection(Driver.java:400)
	at org.postgresql.Driver.connect(Driver.java:259)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$a

In [ ]:
spark.sql("SET spark.sql.yannakakis.countGroupInLeaves = false").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = false").show()
spark.sql("SET spark.sql.yannakakis.enabled = true").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = false").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = true").show()
spark.sql("SET spark.sql.yannakakis.countGroupInLeaves = false").show()

In [ ]:
spark.sql("SET spark.sql.cbo.enabled = true").show()

In [ ]:
spark.sql("ANALYZE TABLE part COMPUTE STATISTICS;").show()

In [ ]:
df = spark.sql("""select count(*) from patents p1, patents p2, patents p3 where p1.toNode = p2.fromNode AND p2.toNode = p3.fromNode""")
df.show()

df.explain(True)

In [ ]:
df = spark.sql("""select avg(i_current_price) from item, inventory where i_item_sk = inv_item_sk""")
df = spark.sql("""select avg(ss_quantity) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select avg(ss_list_price) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select count(isnull(ss_quantity)) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select count(isnull(i_current_price)) from item, inventory where i_item_sk = inv_item_sk""")
#df = spark.sql("""select sum(if (isnull(ss_quantity), 0, ss_quantity)) from store_sales,store where s_store_sk = ss_store_sk""")

df.show()

df.explain(True)

In [ ]:
def run_query(file):
    with open(file, 'r') as f:
        query = '\n'.join(filter(lambda line: not line.startswith('limit') and not line.startswith('-'), f.readlines()))
        
        print("running query: \n" + query)
        return spark.sql(query)

In [ ]:
df = spark.sql("""
SELECT ps_partkey, count(*) from part, partsupp
WHERE p_partkey = ps_partkey
GROUP BY ps_partkey
""")

df.show()

df.explain(True)

In [ ]:
df = spark.sql("SELECT COUNT(p_size) / COUNT(DISTINCT p_size) FROM part")
df.show()
df = spark.sql("SELECT COUNT(p_retailprice) / COUNT(DISTINCT p_retailprice) FROM part")
df.show()

In [ ]:
df_t1 = spark.createDataFrame([(1,1), (2,1), (2,2), (3,2), (3,3), (4,3), (4,3), (5,2), (5,1), (6,4)], schema=("a","b"))
df_t1.createOrReplaceTempView("t1")
df_t2 = spark.createDataFrame([(1,1), (2,1), (3,2), (3,2), (3,3), (3,3), (4,3), (4,2), (5,1), (6,4)], schema=("c","d"))
df_t2.createOrReplaceTempView("t2")
df_t3 = spark.createDataFrame([(1,1), (2,1), (3,2), (3,2), (3,3), (3,3), (4,3), (4,2), (5,1), (6,4)], schema=("e","f"))
df_t3.createOrReplaceTempView("t3")

query = "select median(a) from t1, t2 where b = c"
#query = "select percentile(a, 0.5, b) from t1, t2 where b = c"
#query = "select median(a) from t1 where EXISTS (SELECT 1 FROM t2 WHERE b = c)"
#query = "select count(*) from t1, t2 where b = c"
#query = "select *a from t1 where EXISTS (SELECT 1 FROM t2 WHERE b = c)"

spark.sql("SET spark.sql.yannakakis.enabled = false").show()

df = spark.sql(query)
df.show()

spark.sql("SET spark.sql.yannakakis.enabled = true").show()

df = spark.sql(query)
df.show()

In [ ]:
spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
#spark.conf.set("spark.executor.cores", "6")
#spark.conf.set("spark.executor.instances", "6")
spark.conf.set("spark.sql.shuffle.partitions", "6")

In [ ]:
import pandas as pd
import time

def benchmark(query):    
    spark.sql("SET spark.sql.yannakakis.enabled = true").show()

    start_time = time.time()

    df1 = run_query(query)
    df1.show()
    #df1.explain(True)

    end_time = time.time()
    yannakakis_time = end_time - start_time

    spark.sql("SET spark.sql.yannakakis.enabled = false").show()

    start_time = time.time()

    df2 = run_query(query)
    df2.show()
    #df2.explain(True)

    end_time = time.time()
    ref_time = end_time - start_time
    
    #return [query, ref_time, yannakakis_time]
    return [query, ref_time, yannakakis_time]


queries = ['lsqb/sql/q1.sql', 'lsqb/sql/q3.sql', 'lsqb/sql/q4.sql']

results = [benchmark(q) for q in queries]

df = pd.DataFrame(results, columns = ['query', 'ref_time', 'yannakakis_time'])

print(df)

df.to_csv("results.csv")
    

#print(f'row count: {df1.count()} vs. {df2.count()}' )
    #print(f'time ref: {ref_time}\ntime yannakakis: {yannakakis_time}')

+--------+
|count(1)|
+--------+
|14836038|
+--------+

             query     ref_time  yannakakis_time
0  lsqb/sql/q1.sql    16.608744         5.376228
1  lsqb/sql/q3.sql  4814.826147      4494.824324
2  lsqb/sql/q4.sql     2.609319         2.949713


In [ ]:
#spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
#spark.conf.set("spark.executor.cores", "1")
#spark.conf.set("spark.executor.instances", "1")
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [ ]:
## Compare result
import time
#query = 'tpch-kit/dbgen/queries/postgres/2.sql'
#query = 'tpch-kit/dbgen/queries/postgres/13.sql'
#query = 'count-3.sql'
#query = 'tpch-kit/dbgen/queries/postgres/11.sql'
#query = '11-simple.sql'
query = 'median-2-hint.sql'
#query = 'tpch-kit/dbgen/queries/postgres/7.sql'
#query = '13-simple.sql'
query = 'dsb-queries/agg_queries/query101.sql'
query = 'snap-queries/patents02.sql'
query = 'lsqb/sql/q4.sql'

spark.sql("SET spark.sql.yannakakis.enabled = true").show()

start_time = time.time()

df1 = run_query(query)
df1.show()
df1.explain(mode="extended")

end_time = time.time()
yannakakis_time = end_time - start_time

spark.sql("SET spark.sql.yannakakis.enabled = false").show()

start_time = time.time()

df2 = run_query(query)
df2.show()
df2.explain(mode="extended")

end_time = time.time()
ref_time = end_time - start_time

#print(f'row count: {df1.count()} vs. {df2.count()}' )
print(f'time ref: {ref_time}\ntime yannakakis: {yannakakis_time}')